In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
companiesRaw = pd.read_csv('companies_raw.csv')

/Users/ksopin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
companiesRaw.columns

Index(['CompanyName', ' CompanyNumber', 'RegAddress.CareOf',
       'RegAddress.POBox', 'RegAddress.AddressLine1',
       ' RegAddress.AddressLine2', 'RegAddress.PostTown', 'RegAddress.County',
       'RegAddress.Country', 'RegAddress.PostCode', 'CompanyCategory',
       'CompanyStatus', 'CountryOfOrigin', 'DissolutionDate',
       'IncorporationDate', 'Accounts.AccountRefDay',
       'Accounts.AccountRefMonth', 'Accounts.NextDueDate',
       'Accounts.LastMadeUpDate', 'Accounts.AccountCategory',
       'Returns.NextDueDate', 'Returns.LastMadeUpDate',
       'Mortgages.NumMortCharges', 'Mortgages.NumMortOutstanding',
       'Mortgages.NumMortPartSatisfied', 'Mortgages.NumMortSatisfied',
       'SICCode.SicText_1', 'SICCode.SicText_2', 'SICCode.SicText_3',
       'SICCode.SicText_4', 'LimitedPartnerships.NumGenPartners',
       'LimitedPartnerships.NumLimPartners', 'URI', 'PreviousName_1.CONDATE',
       ' PreviousName_1.CompanyName', ' PreviousName_2.CONDATE',
       ' PreviousName_2.C

In [12]:
def dstrtiyint(x):
    atetime_object = datetime.strptime(x, '%d/%m/%Y')
    return atetime_object.year

def sicCode(x):
    return x.split(" ")[0]

def sicDescription(x):
    els = x.split("-")
    return els[len(els)-1].strip()

def sicSector(x):
    return x[0:2]

companies = companiesRaw[['RegAddress.PostTown',                           
                          'CompanyCategory', 'CountryOfOrigin', 
                          'CompanyStatus', 'IncorporationDate',
                          'SICCode.SicText_1'
                          ]]


companies['Year'] = companies['IncorporationDate'].apply(dstrtiyint)
companies = companies[companies['Year'] > 1970]



companies['Date'] = pd.to_datetime(companies['IncorporationDate'], format='%d/%m/%Y')
companies = companies.drop(columns=['IncorporationDate'])
companies = companies.rename(columns={"RegAddress.PostTown": "City", "CompanyCategory": "Type",
                         "CompanyStatus":"Status", "SICCode.SicText_1":"Sic"})
companies['SicCode'] = companies['Sic'].apply(sicCode)
companies['SicSector'] = companies['SicCode'].apply(sicSector)
companies['SicDescription'] = companies['Sic'].apply(sicDescription)
companies = companies.drop(columns=['Sic'])
companies.head()



<ipython-input-12-1e48b195171e>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies['Year'] = companies['IncorporationDate'].apply(dstrtiyint)


,City,Type,CountryOfOrigin,Status,Year,Date,SicCode,SicSector,SicDescription
0,SHEFFIELD,Private Limited Company,United Kingdom,Active,2020,2020-07-29,78300,78,Human resources provision and management of hu...
1,LEEDS,Private Limited Company,United Kingdom,Active,2012,2012-09-11,99999,99,Dormant Company
2,ROMILEY,Private Limited Company,United Kingdom,Active,2018,2018-06-05,47710,47,Retail sale of clothing in specialised stores
3,LONDON,Private Limited Company,United Kingdom,Active,2018,2018-12-28,18129,18,Printing n.e.c.
4,CHINGFORD,Private Limited Company,United Kingdom,Active,2020,2020-01-14,90030,90,Artistic creation


In [9]:
companies.dtypes

City                       object
Type                       object
CountryOfOrigin            object
Status                     object
Year                        int64
Date               datetime64[ns]
SicCode                    object
SicSector                  object
dtype: object

### Save Data to file

In [13]:
companies.to_csv('companies.csv', index=False)